# Summary
#  [Importing libraries](#Importing-libraries)
#  [Initial configuration](#Initial-configuration)
#  [Importing Excel spreadsheet](#Defining-functions)
#  [Importing Excel spreadsheet](#Importing-Excel-spreadsheet)
# [Setting up a local copy of the NCBI taxonomy database](#Setting-up-a-local-copy-of-the-NCBI-taxonomy-database)
# [Geting NCBI TAX IDs](#Geting-NCBI-TAX-IDs)
# [Creating dataframe with all microorganisms in the rows and the plastics in the columns](#Creating-dataframe-with-all-microorganisms-in-the-rows-and-the-plastics-in-the-columns)
# [Filling the dataframe with plastic degrading data](#Filling-the-dataframe-with-plastic-degrading-data)
# [Getting tree topology from NCBI and creating the tree](#Getting-tree-topology-from-NCBI-and-creating-the-tree)
# [Creating Itol dataset files](#Creating-Itol-dataset-files)
# [Downloading all available genomes for the plastic degraders](#Downloading-all-available-genomes-for-the-plastic-degraders)
# [Creating itol dataset for microorganisms with available genomes](#Creating-itol-dataset-for-microorganisms-with-available-genomes)
# [Blasting genes against genomes](#Blasting-genes-against-genomes)
# [Creating itol datasets for genes](#Creating-itol-datasets-for-genes)

<a id='Importing-libraries'></a>
# Importing libraries

In [1]:
import pandas as pd
import random, os, time
from ete3 import NCBITaxa
from ete3 import Tree

<a id='Initial-configuration'></a>
# Initial configuration

In [2]:
#If running on a Slurm cluster, change account name and change run_on_slurm variable to True.
account = "uoa00354"
run_on_slurm = True

<a id='Defining-functions'></a>
# Defining functions

In [3]:
def create_directory(directory_name):
    if os.path.exists(directory_name) == False:
        os.mkdir(directory_name)
    
def run_command(run_on_slurm, account, job_name, job_time, job_memory, job_cpus, job_command):
    if run_on_slurm == False:
        os.system(job_command)
    if run_on_slurm == True:
        f = open("run.sh", "w")
        f.write("""#!/bin/bash

#SBATCH --account="""+str(account)+"""
#SBATCH --job-name="""+str(job_name)+"""
#SBATCH --time="""+str(job_time)+"""
#SBATCH --mem="""+str(job_memory)+"""
#SBATCH --cpus-per-task="""+str(job_cpus)+"""

module load Python
module load BLAST

"""+job_command)
        f.close()
    
        os.system("sbatch run.sh")  
        os.system("rm run.sh")
    else:
        print("Please, set 'run_on_slurm' variable to True or False")

<a id='Importing-Excel-spreadsheet'></a>
# Importing Excel spreadsheet

In [4]:
df = pd.read_excel("Degraders_list.xlsx")
#Visualize part of the dataset.
df.head(7)

,Microorganism,Tax ID,Plastic,Ref,Enzyme,Gene,Sequence
0,Absidia sp.,0,PHB,"Jeszeová, L., Puškárová, A., Bučková, M., Krak...",No,No,NaN
1,Absidia sp.,0,PLA,"Jeszeová, L., Puškárová, A., Bučková, M., Krak...",No,No,NaN
2,Achromobacter xylosoxidans,0,HDPE,"Kowalczyk, A., Chyc, M., Ryszka, P., & Latowsk...",No,No,NaN
3,Acidovorax avenae,0,PCL,"Mergaert, J., Ruffieux, K., Bourban, C., Storm...",No,No,NaN
4,Acidovorax avenae,0,PBSA,"Mergaert, J., Ruffieux, K., Bourban, C., Storm...",No,No,NaN
5,Acidovorax delafieldii,0,PHBH,"Morohoshi, T., Oi, T., Aiso, H., Suzuki, T., O...",No,No,NaN
6,Acidovorax delafieldii,0,PBSA,"Uchida, H., Nakajima-Kambe, T., Shigeno-Akutsu...",Lipase,No,NaN


<a id='Setting-up-a-local-copy-of-the-NCBI-taxonomy-database'></a>
# Setting up a local copy of the NCBI taxonomy database

In [5]:
ncbi = NCBITaxa()

<a id='Geting-NCBI-TAX-IDs'></a>
# Geting NCBI TAX IDs

In [6]:
#If it returns an error, just wait, it will get the dataframe anyway.
for index, row in df.iterrows():
    if row["Tax ID"] == 0:
        name2taxid = ncbi.get_name_translator([row["Microorganism"]])
        df["Tax ID"][row.name] = str(name2taxid.values()).replace("[","").replace("]","").split("(")[1].split(")")[0]
df.head(7)

/home/vmar559/PyPackages/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Microorganism,Tax ID,Plastic,Ref,Enzyme,Gene,Sequence
0,Absidia sp.,1982014,PHB,"Jeszeová, L., Puškárová, A., Bučková, M., Krak...",No,No,NaN
1,Absidia sp.,1982014,PLA,"Jeszeová, L., Puškárová, A., Bučková, M., Krak...",No,No,NaN
2,Achromobacter xylosoxidans,85698,HDPE,"Kowalczyk, A., Chyc, M., Ryszka, P., & Latowsk...",No,No,NaN
3,Acidovorax avenae,80867,PCL,"Mergaert, J., Ruffieux, K., Bourban, C., Storm...",No,No,NaN
4,Acidovorax avenae,80867,PBSA,"Mergaert, J., Ruffieux, K., Bourban, C., Storm...",No,No,NaN
5,Acidovorax delafieldii,47920,PHBH,"Morohoshi, T., Oi, T., Aiso, H., Suzuki, T., O...",No,No,NaN
6,Acidovorax delafieldii,47920,PBSA,"Uchida, H., Nakajima-Kambe, T., Shigeno-Akutsu...",Lipase,No,NaN


<a id='Creating-dataframe-with-all-microorganisms-in-the-rows-and-the-plastics-in-the-columns'></a>
# Creating dataframe with all microorganisms in the rows and the plastics in the columns

In [7]:
plastic_types = df['Plastic'].unique()
tax_ids = df['Tax ID'].unique()
df2 = pd.DataFrame(0, index=tax_ids, columns=plastic_types)
df2.sort_index(inplace=True)
df2.sort_index(axis=1, inplace=True)
df2.head(7)

,BTA copolyester,Ecoflex,HDPE,LDPE,LDPE Blend,Mater-Bi,Nylon,Nylon 12,Nylon 4,Nylon 6,...,PTMS,PTS,PU,PU Blend,PVA-LLDPE,PVC,PVC Blend,Poly(3HB-co-4HB),Polyether sulfone,Sky-Green
232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
287,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<a id='Filling-the-dataframe-with-plastic-degrading-data'></a>
# Filling the dataframe with plastic degrading data

In [8]:
references_dict = {}
for index, row in df.iterrows():
    plastic = row['Plastic']
    tax_id = row['Tax ID']
    reference = str(row['Ref'].encode('utf-8').strip())
    df2.at[tax_id, plastic] = 1
    if tax_id not in references_dict.keys():
        references_dict[tax_id] = reference
    else:
        references_dict[tax_id] += reference
df2.head(7)

,BTA copolyester,Ecoflex,HDPE,LDPE,LDPE Blend,Mater-Bi,Nylon,Nylon 12,Nylon 4,Nylon 6,...,PTMS,PTS,PU,PU Blend,PVA-LLDPE,PVC,PVC Blend,Poly(3HB-co-4HB),Polyether sulfone,Sky-Green
232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
287,0,0,1,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<a id='Getting-tree-topology-from-NCBI-and-creating-the-tree'></a>
# Getting tree topology from NCBI and creating the tree

In [14]:
ids = df2.index.values
t = ncbi.get_topology(ids, intermediate_nodes=True)
'''
leaves = t.get_leaves()
for n in range(0, len(leaves)):
    tax_id = int(leaves[n].name)
    reference = references_dict[tax_id]
    leaves[n].add_feature("References", reference)
    taxid_to_sci_name_dict[leaves[n].name] = leaves[n].sci_name
    leaves[n].name = leaves[n].sci_name
'''
t.write(format=8, outfile="tree.nw")


<a id='Creating-dictionary-with-organism-names-to-TAX-IDs'></a>
# Creating dictionary with organism names to TAX IDs

In [119]:
tax_id_to_organism_name_dict = {}
for index, row in df2.iterrows():
    tax_id_to_organism_name_dict[index] = taxid_to_sci_name_dict[str(index)].replace("(","_").replace(":","_").replace(")","_").replace("[","_").replace("]","_")

<a id='Creating-Itol-dataset-files'></a>
# Creating Itol dataset files

In [16]:
r = lambda: random.randint(0,255) #Creates random hexadecimal color codes
create_directory("itol_datasets")

for column in df2.columns:
    f = open("itol_datasets/"+column.rstrip()+".txt", "w")
    f.write("""DATASET_BINARY
SEPARATOR COMMA
DATASET_LABEL,"""+column+"""
COLOR,"""+'#%02X%02X%02X' % (r(),r(),r())+"""
FIELD_SHAPES,1
FIELD_LABELS,f1
DATA""")
    for index, row in df2.iterrows():
        tax_id = str(index)
        feature_binary = str(row[column])
        f.write("\n"+tax_id+","+feature_binary)
    f.close()

<a id='Downloading-all-available-genomes-for-the-plastic-degraders'></a>
# Downloading all available genomes for the plastic degraders

In [45]:
for tax_id in tax_ids:
    job_name = str(tax_id)+"_download"
    job_time = "05:00:00"
    job_memory = 100 #Amount of ram memory in MB
    job_cpus = 1
    job_command = "ncbi-genome-download -s genbank --format fasta --taxid "+str(tax_id)+" -r 5 -o genomes/"+str(tax_id)+"/ fungi,bacteria"
    run_command(run_on_slurm, account, job_name, job_time, job_memory, job_cpus, job_command)
    time.sleep(2)

<a id='Creating-itol-dataset-for-microorganisms-with-available-genomes'></a>
# Creating itol dataset for microorganisms with available genomes

In [157]:
create_directory("itol_datasets_genomes")
directories_list = os.listdir("genomes")

f = open("itol_datasets_genomes/Available_genomes.txt", "w")
f.write("""DATASET_BINARY
SEPARATOR COMMA
DATASET_LABEL,Available genomes
COLOR,#FF0000
FIELD_SHAPES,1
FIELD_LABELS,f1
DATA""")

for index, row in df2.iterrows():
    organism_name = tax_id_to_organism_name_dict[index]
    if str(index) in directories_list:
        f.write("\n"+organism_name+","+"1")
    else:
        f.write("\n"+organism_name+","+"0")
f.close()

<a id='Blasting-genes-against-genomes'></a>
# Blasting genes against genomes

In [126]:
"""

#Adding the TAX ID to all sequence names
create_directory("genomes_database")
os.system("touch genomes_database/genomes.fasta")
for directory in directories_list:
    os.system("zcat genomes/"+directory+"/genbank/*/*/*genomic.fna.gz | sed s/'^>'/'>"+directory+"_"+"'/g >> genomes_database/genomes.fasta")

#Creating BLAST database
os.system("makeblastdb -in genomes_database/genomes.fasta -dbtype nucl")

"""

#Sending BLAST jobs
create_directory("blasts")
job_time = "01:00:00"
job_memory = 512 #Amount of ram memory in MB
job_cpus = 1
for index, row in df.iterrows():
    sequences = row["Sequence"]
    if pd.notna(sequences) == True:
        for n in range(0, len(sequences.split(";"))):
            species_name = tax_id_to_organism_name_dict[row["Tax ID"]].replace(" ","_")
            job_name = str(row["Tax ID"])+"_"+str(n)
            job_command = "echo '"+sequences.split(";")[n]+"' | tblastn -query - -db genomes_database/genomes.fasta -out blasts/"+species_name+"_"+row["Enzyme"].replace(" ","_")+"_"+str(n)+".tsv -outfmt 6"
            run_command(run_on_slurm, account, job_name, job_time, job_memory, job_cpus, job_command)
            time.sleep(2)


<a id='Creating-itol-datasets-for-all-genes'></a>
# Creating itol datasets for all genes

In [155]:
r = lambda: random.randint(0,255) #Creates random hexadecimal color codes

blast_outputs_list = os.listdir("blasts")
for file in blast_outputs_list:
    file_blast = open("blasts/"+file)
    blast = file_blast.readlines()
    file_itol = open("itol_datasets_genomes/"+file.replace("tsv", "txt"), "w")
    counting_dict = {}
    for line in blast:
        qseqid = line.rstrip().split("\t")[0]
        sseqid = line.rstrip().split("\t")[1].split("_")[0]
        pident = float(line.rstrip().split("\t")[2])
        length = int(line.rstrip().split("\t")[3])
        mismatch = int(line.rstrip().split("\t")[4])
        gapopen = int(line.rstrip().split("\t")[5])
        qstart = int(line.rstrip().split("\t")[6])
        qend = float(line.rstrip().split("\t")[7])
        sstart = float(line.rstrip().split("\t")[8])
        send = float(line.rstrip().split("\t")[9])
        evalue = float(line.rstrip().split("\t")[10])
        bitscore = float(line.rstrip().split("\t")[11])
        if evalue <= 1e-06 and pident >= 25:
            if sseqid not in counting_dict:
                counting_dict[sseqid] = 1
            else:
                counting_dict[sseqid] += 1
                
    file_itol.write("""DATASET_MULTIBAR
SEPARATOR TAB
DATASET_LABEL	"""+file.replace('.tsv', '').replace('_', ' ')+"""
COLOR	"""+'#%02X%02X%02X' % (r(),r(),r())+"""
FIELD_COLORS	"""+'#%02X%02X%02X' % (r(),r(),r())+"""
FIELD_LABELS	"""+file.replace('.tsv', '').replace('_', ' ')+"""
DATA""")    
    for taxid, organism_name in tax_id_to_organism_name_dict.items():
        if str(taxid) in counting_dict:
            file_itol.write("\n"+organism_name+"\t"+str(counting_dict[str(taxid)]))
        else:
            file_itol.write("\n"+organism_name+"\t0")
    file_blast.close()
    file_itol.close()

<a id='Creating-itol-datasets-for-phylum-labels'></a>
# Creating itol datasets for phylum labels

In [64]:
f = open("itol_datasets/Phylum_labels.txt", "w")
f.write("""DATASET_TEXT
SEPARATOR COMMA
DATASET_LABEL,example text dataset
COLOR,#ff0000
MARGIN,0
SHOW_INTERNAL,1
LABEL_ROTATION,0
STRAIGHT_LABELS,0
ALIGN_TO_TREE,1
SIZE_FACTOR,1
DATA
""")

organism_phylum_dict = {}
organism_class_dict = {}

for index, row in df2.iterrows():
    lineage = ncbi.get_lineage(index)
    for item in lineage:
        if "'phylum'" in str(ncbi.get_rank([item])):
            organism_phylum = ncbi.get_taxid_translator([item])[item]
        if "'class'" in str(ncbi.get_rank([item])):
            organism_class = ncbi.get_taxid_translator([item])[item]
    if organism_class not in organism_class_dict:
        organism_class_dict[organism_class] = [item]
        organism_phylum_dict[organism_phylum] = [organism_class]
    else:
        organism_class_dict[organism_class].append(item)
        if organism_phylum in organism_phylum_dict:
            organism_phylum_dict[organism_phylum].append(organism_class)
        else:
            organism_phylum_dict[organism_phylum] = [organism_class]
for key, value in organism_phylum_dict.items():
    for key2, value2 in organism_class_dict.items():
        print(key2, value2)
        if len(value2) > 1:
            f.write("\n"+str(value2[0])+"|"+str(value2[1])+","+str(key)+",-1,"+'#%02X%02X%02X' % (r(),r(),r())+",bold,10,0")
        else:
            f.write("\n"+str(value2[0])+"|"+str(value2[0])+","+str(key)+",-1,"+'#%02X%02X%02X' % (r(),r(),r())+",bold,10,0")
f.close()

Gammaproteobacteria [232, 287, 294, 300, 303, 306, 316, 317, 470, 471, 479, 549, 573, 615, 663, 670, 29446, 34038, 35703, 40215, 40324, 42895, 43263, 47877, 47878, 47881, 47886, 48296, 50422, 53408, 56811, 61645, 61652, 69392, 69393, 76759, 76760, 76761, 78556, 108980, 137658, 202952, 226011, 254161, 319939, 380021, 405444, 477689, 570156, 587753, 676599, 693231, 1148157, 1407621, 1869214, 1871049, 1872427]
Flavobacteriia [239]
Betaproteobacteria [285, 292, 329, 511, 512, 12917, 28066, 28095, 29443, 34028, 34030, 34073, 36773, 47920, 54061, 60552, 65048, 75659, 76731, 80866, 80867, 85698, 94624, 106590, 180282, 215580, 401470, 592050, 1547922, 1872122, 1872437, 1873897, 1886637, 1934313]
Alphaproteobacteria [358, 1673, 42190, 94625, 335286]
Oligoflexia [959]
Actinobacteria [1270, 1271, 1273, 1667, 1720, 1785, 1814, 1816, 1824, 1829, 1830, 1831, 1833, 1852, 1877, 1881, 1911, 1931, 1934, 1941, 1952, 1989, 2017, 2021, 2030, 2065, 2072, 31958, 33907, 33910, 33919, 33920, 33921, 36819, 3763